In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing import image

In [ ]:
traincsv=pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")
test="../input/sartorius-cell-instance-segmentation/test"
train="../input/sartorius-cell-instance-segmentation/train"
train_supervised="../input/sartorius-cell-instance-segmentation/train_semi_supervised"
print("Shape: ", traincsv.shape)
traincsv.tail(30)

In [ ]:
traincsv.info()

In [ ]:
traincsv.describe()

In [ ]:
print("length of test: ", len(test))
print("length of train: ", len(train))
print("\nNumber of unique values:")
traincsv.nunique()

In [ ]:
import seaborn as sns
ax = sns.countplot(x="cell_type", data=traincsv, palette="Set3")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (15,8))
ax = sns.countplot(x="plate_time", data=traincsv, palette="Set2",)


In [ ]:
plt.figure(figsize = (20,8))
ax = sns.countplot(x="sample_date", data=traincsv, palette="dark")

In [ ]:
plt.figure(figsize = (20,8))
ax = sns.countplot(x="elapsed_timedelta", data=traincsv, palette="colorblind",)

In [ ]:
from keras.preprocessing import image    
im1_path="../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png"
test_image=image.load_img(im1_path,target_size=(256,256))

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
from PIL import Image
assert os.path.isdir(train) == True
sample_train_images = list(os.walk(train))[0][2][:8]
sample_train_images = list(map(lambda x: os.path.join(train, x), sample_train_images))
plt.figure(figsize = (17,17))
for iterator, filename in enumerate(sample_train_images):
    image = Image.open(filename)
    plt.subplot(4,2,iterator+1)
    plt.imshow(image,cmap='bone_r')
plt.tight_layout()

In [ ]:
assert os.path.isdir(test) == True
sample_test_images = list(os.walk(test))[0][2][:8]
sample_test_images = list(map(lambda x: os.path.join(test, x), sample_test_images))
plt.figure(figsize = (17,17))
for iterator, filename in enumerate(sample_test_images):
    image = Image.open(filename)
    plt.subplot(4,2,iterator+1)
    plt.imshow(image,cmap='bone_r')
plt.tight_layout()

In [ ]:
assert os.path.isdir(train_supervised) == True
sample_trainsemi_images = list(os.walk(train_supervised))[0][2][:8]
sample_trainsemi_images = list(map(lambda x: os.path.join(train_supervised, x), sample_trainsemi_images))
plt.figure(figsize = (17,17))
for iterator, filename in enumerate(sample_trainsemi_images):
    image = Image.open(filename)
    plt.subplot(4,2,iterator+1)
    plt.imshow(image,cmap='bone_r')
plt.tight_layout()

In [ ]:
! pip install segmentation-models

In [ ]:
! pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
from keras import backend as K
from keras.losses import binary_crossentropy
from segmentation_models.losses import bce_jaccard_loss
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

import tensorflow as tf
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + 0.5 * dice_loss(tf.cast(y_true, tf.float32), y_pred)
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

from segmentation_models import Unet
from segmentation_models.utils import set_trainable


model = Unet('efficientnetb7',input_shape=(512, 512, 3), classes=3, activation='sigmoid',encoder_weights='imagenet')
model.compile(optimizer='adam', loss=bce_dice_loss,metrics=[dice_coef,iou_coef,'accuracy']) #bce_dice_loss binary_crossentropy
model.summary()

# Please Upvote, Work in progress